In [ ]:
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user

In [ ]:
!python --version

In [ ]:
!pip install scipy==1.8.1


In [ ]:
!pip install Cython==0.29.35

In [ ]:
!pip install scikit-learn==0.24.2 --no-build-isolation

In [ ]:
!pip install auto-sklearn

In [ ]:
!pip show auto-sklearn

In [ ]:
!pip install tabulate

In [ ]:
import autosklearn.classification as classification
import autosklearn.pipeline as pipeline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
from score import pipeline_score
import os
import io
import sys
import csv

In [ ]:
# Redireciona os prints da função
def capturar_e_salvar_prints_em_csv(func, *args, csv_filename="log.csv", **kwargs):
    buffer = io.StringIO()
    stdout_original = sys.stdout  # Guarda o stdout original
    sys.stdout = buffer           # Redireciona para o buffer

    try:
        resultado = func(*args, **kwargs)
    finally:
        sys.stdout = stdout_original  # Restaura o stdout original

    # Pega o conteúdo dos prints
    saida = buffer.getvalue().splitlines()

    # Salva em um CSV
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["mensagem"])
        for linha in saida:
            writer.writerow([linha])

    return resultado

# Listar todos os arquivos CSV na pasta
datasets = [f for f in os.listdir() if f.endswith(".csv")]

# Caminho do arquivo CSV onde os resultados serão armazenados
result_csv_path = "AutoSklearn_results.csv"

# Se o arquivo de resultados ainda não existir, cria com cabeçalho
if not os.path.exists(result_csv_path):
    pd.DataFrame(columns=["dataset", "best_params", "test_score", "execution_time"]).to_csv(result_csv_path, index=False)

model = classification.AutoSklearnClassifier(include={'classifier': ['decision_tree','k_nearest_neighbors','libsvm_svc', 'random_forest']})

for dataset in datasets:
  if dataset != "AutoSklearn_results.csv":
    print(f"\nProcessando: {dataset}")

    # Carregar o dataset
    df = pd.read_csv(dataset)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    execution_time = end_time - start_time

    print(model.leaderboard())

    y_pred = model.predict(X_test)
    final_score = capturar_e_salvar_prints_em_csv(pipeline_score, y_test, y_pred, verbosity=True, csv_filename=f"tabela {dataset}")

    # Criar um dicionário com os resultados
    result = {
        "dataset": dataset,
        # Convertendo para string para evitar problemas de formatação
        "best_params": str(model.show_models()),
        "test_score": final_score,
        "execution_time": execution_time,
    }

    # Adicionar ao arquivo CSV imediatamente
    pd.DataFrame([result]).to_csv(result_csv_path, mode="a", header=False, index=False)


Processando: Sepsis Survival Minimal Clinical Records.csv
[WARNING] [2025-04-25 02:12:04,330:Client-AutoMLSMBO(1)::ac637a19-217a-11f0-94e7-0242ac1c000c] Configuration 232 not found
[WARNING] [2025-04-25 02:12:04,330:Client-AutoMLSMBO(1)::ac637a19-217a-11f0-94e7-0242ac1c000c] Configuration 422 not found
[WARNING] [2025-04-25 02:12:04,330:Client-AutoMLSMBO(1)::ac637a19-217a-11f0-94e7-0242ac1c000c] Configuration 310 not found
[WARNING] [2025-04-25 02:12:04,330:Client-AutoMLSMBO(1)::ac637a19-217a-11f0-94e7-0242ac1c000c] Configuration 437 not found
[WARNING] [2025-04-25 02:12:04,330:Client-AutoMLSMBO(1)::ac637a19-217a-11f0-94e7-0242ac1c000c] Configuration 69 not found
[WARNING] [2025-04-25 02:12:04,330:Client-AutoMLSMBO(1)::ac637a19-217a-11f0-94e7-0242ac1c000c] Configuration 653 not found
[WARNING] [2025-04-25 02:12:04,330:Client-AutoMLSMBO(1)::ac637a19-217a-11f0-94e7-0242ac1c000c] Configuration 40 not found
[WARNING] [2025-04-25 02:12:04,330:Client-AutoMLSMBO(1)::ac637a19-217a-11f0-94e7-0